# k邻近法（KNN）


## k临近算法
输入：训练数据集
$$
T=\{ (x_1,y_1),(x_1,y_1),\dots ,(x_n,y_n)\}
$$
其中，$x_i \in \chi \subseteq R^n$为特征向量， $y_i \in Y=\{c_1,c_22,\cdots ,c_k\}$为实例的类别;实例特征向量x

输出:x所属的类y

(1)根据给定的距离度量，在训练集T中找出与x最邻近的k个点，涵盖这k个点的x的邻域记作$N_k(x)$

(2)在$N_k(x)$中根据分类决策规则(如多数表决)决定x的类别y:
$$
y=\operatorname{arg}\max_{c_j}\sum_{x_i\in N_k(x)}\operatorname{I}(y_i=c_j);\quad j=1,2,\cdots,k
$$
其中$\operatorname{I}$为指示函数，当$y_i=c_j$时值为1，否则为0

## k邻近模型
### 距离度量
距离的度量函数:一般的度量函数为$L_p$距离或$\operatorname{Minkowski}$距离。
设特征空间$\operatorname{X}$是n维实数向量空间，$x_i=(x_i^{(1)},x_i^{(2)},\cdots,x_i^{(n)})^T$,$x_j=(x_j^{(1)},x_j^{(2)},\cdots,x_j^{(n)})^T$,$x_i,x_j$的$L_p$距离定义为
$$
L_p(x_i,x_j)=\left(\sum_{l=1}^n\bracevert x_i^{(l)}-x_j^{(l)}\bracevert^p \right)^{\frac{1}{p}}
$$
当p=2时，称为欧式距离

当p=1时，称为曼哈顿距离

当p=$\infty$时，是各个坐标距离的最大值
### k值选择
k值较小相当于使用较小的邻域中的训练实例进行预测，学习的近似误差会减小，但估计误差会增大，预测结果对近邻的实例点敏感，使得模型更复杂，容易发生过拟合。

k值较大相当于使用较大的邻域中的训练实例进行预测，学习的估计误差会减小，但近似误差会增大，使得模型变得简单，容易发生欠拟合。
P.S.近似误差更关注训练，可以理解为对训练集的训练误差。

   估计误差更关注测试，泛化，可以理解为对测试集的测试误差。
   
   近似误差和估计误差不可得兼，实际一般采用交叉验证法来选取最优k值。






## k邻近法的实现：kd树
### 构造kd树
kd树是二叉树，表示对k维空间的一个划分。构造kd树相当于不断地用垂直于坐标轴的超平面将k维空间切分。 

kd树构造算法：

输入:k维空间数据集$k=\{x_1,x_2,\cdots x_n\}$,其中$x_i=(x_i^{(1)},x_i^{(2)},\cdot,x_i^{(k)})^T$

输出:kd树

(1)开始:构造根节点。选择$x^{(1)}$为坐标轴，以T中所有实例的$x^{(1)}$坐标的中位数为切分点，将根节点对应的超矩形区域切分为两个子区域。由根节点生成深度为1的左、右子节点，左子节点对应坐标$x^{(1)}$小于切分点的子区域，右子节点对应坐标$x^{(1)}$大于切分点的子区域。

(2)重复：对于深度为j的节点取$x^{(l)}$为坐标轴，l=j(modk)+1,将该节点对应的超平面切分为连个子区域。

(3)直到两个子区域没有实例存在时停止。

In [5]:
#kd树构建$x^{(1)}$为坐标轴
class kdNode(object):
    def __init__(self, split, dim, left, right):
        self.split = split #分割的样本点
        self.dim = dim     #分割对应维度
        self.left = left   #左子区域
        self.right = right#右子区域

class kdTree(object):
    def __init__(self, data):
        k = len(data[0])
        
        def CreateNode(dim, data_set=None):
            if not data_set:return None
            data_set.sort(key = lambda x:x[dim])
            pos = len(data_set)//2
            split = data_set[pos]
            next_dim = (dim + 1)%k
            return kdNode(
                split,
                dim,
                CreateNode(next_dim, data_set[:pos]),
                CreateNode(next_dim, data_set[pos+1:])
            )
        self.root = CreateNode(0, data)

### kd树的最邻近搜索
输入:已构造的kd树;目标点x;

输出:x的最邻近点

(1)在kd树中找到包含目标点x的叶节点：从根节点出发，递归地向下访问kd树，若目标节点x的当前维小于切分点坐标，则移动到左节点，否则移动到右节点,直到当前节点为叶节点为止

(2)此叶节点为“当前最近点”。

(3)递归地向上回退，在每个节点进行如下操作：

(a)若该节点保存的实例点距离目标点x距离更近，则该实例点为“当前最近点”

(b)当前最近点一定存在于该节点一个子结点对应的区域，检查该子节点的父节点的另一子结点对应的区域是否有更近的点，具体地检查另一子结点对应的区域是否与目标点为球心，以“当前最近点”与目标点之间距离为半径的超球体相交。

如果相交，则移动到另一子结点，否则向上回退。

(4)回退到根节点时，“当前最近点”即为x的最临近点

In [23]:
import math
class result(object):
    def __init__(self, nearest_dist, nearest_point, num_node):
        self.nearest_dist = nearest_dist
        self.nearest_point = nearest_point
        self.num_node = num_node
def find(kd, x):
    k = len(x)
    
    def check(kd_node, target, mindist):
        #输入kd树的节点， 目标点，当前最小距离；输出 当前最小距离对应的node，当前最小距离，已访问的节点数（用来衡量效率，算法本身没有要求）
        if not kd_node: return result(float("inf"), [0]*k, 0)
        visited = 1
        dim = kd_node.dim
        split = kd_node.split
        
        if target[dim]<split[dim]:
            next_node = kd_node.left
            neighbor = kd_node.right
        else:
            next_node = kd_node.right
            neighbor = kd_node.left
        
        tmp1 = check(next_node, target, mindist)
        #算法第一步， 递归找到叶节点
        
        point = tmp1.nearest_point
        dist = tmp1.nearest_dist        
        visited += tmp1.num_node
        #第二步，以此叶节点为当前最近点
        
        if dist < mindist:
            mindist = dist
        
        intersect = abs(split[dim] - target[dim])
        if intersect > mindist:
            return result(dist, point,visited)
        #(3.a)不相交则回退
        """
        这里注明一下，kd树中叶节点对应的是一个子空间，而非叶节点对应的是一个超平面（可见书中图3.5）
        因此，计算当前节点到超平面的距离只要考虑一个维度即可
        若当前结点对应超球体与超平面不相交，则显然超平面对应点到当前节点的距离要大于超球体的半径，因此可以不用判断
        若相交，则要考虑超平面对应节点及其另一个子节点对应区域
        因此，算法的具体顺序为：
        (1)判断超球体与超平面是否相交，不相交则回退
        (2)检查当前节点到当前最近点的距离，判断是否更新
        (3)递归检查当前节点的另一节点
        """
        tmp_dist = math.sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(target, split)))
        if tmp_dist < dist:
            point = split
            dist = tmp_dist
            mindist = tmp_dist
            
        tmp2 = check(neighbor, target, mindist)
        visited += tmp2.num_node
        if tmp2.nearest_dist < dist:
            point = split
            dist = tmp_dist
        
        return result(dist, point, visited)
    return check(kd.root, x, float("inf"))